In [1]:
import rebound
import time
import sys
import numpy as np
from matplotlib import pyplot as plt
from astropy import units as u
from astropy import constants as c

# local imports
import heartbeat

In [ ]:
# global variables

glob_dclo = 1. #CE distance to check in rH
dir = 'planet_test/'
glob_archive = ''
glob_names = []
glob_is_close = False

Nsys = 1

# observational data for EPIC 220208795 from van der Kamp et al (2021)
Mstar = 0.85 * u.Msun
Rstar = 0.830 * u.Rsun
name_star = 'EPIC220208795'
# present-day disc object
Pdisc = 290 * u.d
edisc = 0.72
adisc = ((c.G*Mstar*Pdisc**2/(4*np.pi**2))**(1/3)).to(u.au)

rng = np.random.default_rng(12676)

# initial planet architecture
Npl = 2
Rpl = 1 * u.Rjup * np.ones((Nsys,Npl))
Mpl = 2 * u.Mjup * np.ones((Nsys,Npl))
a1 = adisc*2
rH = (a1 * (Mpl[0,0]/(3*Mstar))**(1/3)).to(u.au)
delta = rng.uniform(1,5,Nsys)
apl = np.zeros((Nsys,Npl)) * u.au
apl[:,0] = a1
apl[:,1] = a1 + rH*delta
epl = rng.uniform(0,0.1,(Nsys,Npl))
ipl = np.radians(rng.uniform(0,3,(Nsys,Npl)))
wpl = np.radians(rng.uniform(0,360,(Nsys,Npl))) #arg of peri
Opl = np.radians(rng.uniform(0,360,(Nsys,Npl)))
mapl = np.radians(rng.uniform(0,360,(Nsys,Npl)))
name_pl = ['Planet1','Planet2']

glob_names = [name_star] + name_pl

# times to run for
tmax = 1e6
TINY = 1e-3
t_rewind = 1
tmoons = 1e3

In [ ]:
# setup simulations
def run_sim_pl(i):
    
    global glob_archive
    global glob_is_close
    global glob_is_eject
    global glob_is_stop
    global glob_dclo
    global glob_is_done
    
    print(f'Simulation {i}:')
    
    glob_archive = dir + f"planets_{i:04d}.bin"
    glob_is_close = False
    glob_is_eject = False
    glob_is_stop = False

    try:
        sim = rebound.Simulation(glob_archive)
    except:
        print('Creating new simulation...')
        sim = rebound.Simulation()
        sim.integrator = "ias15"
        sim.units = ('yr', 'AU', 'Msun')

        sim.exit_max_distance = 50000
        sim.track_energy_offset = 0

        sim.heartbeat = heartbeat


        sim.automateSimulationArchive(glob_archive, interval=1000.,deletefile=True)

        sim.add(m=Mstar.to_value(u.Msun),r=Rstar.to_value(u.au),hash=name_star)
        for j in range(Npl):
            sim.add(a=apl[i,j].to_value(u.au),e=epl[i,j],inc=ipl[i,j],omega=wpl[i,j],Omega=Opl[i,j],M=mapl[i,j],
                    m=Mpl[i,j].to_value(u.Msun),r=Rpl[i,j].to_value(u.au),primary=sim.particles[name_star],
                    hash=name_pl[j])

        sim.move_to_com()
        
    else:
        print(f'Restoring from file... t = {sim.t} years')
#check end state
        if len(sim.particles) == 2:
            print('restored simulation has only one planet')
            print()
            glob_is_eject = True
        for p in sim.particles[1:]:
            rh2 = p.d**2 * (p.m/(3*sim.particles[0].m))**(2/3)
            for q in sim.particles[1:]:
                if p.hash.value == q.hash.value:
                    continue
                dx = p.x-q.x
                dy = p.y-q.y
                dz = p.z-q.z
                d2 = dx*dx + dy*dy + dz*dz
                if d2 <= rh2*glob_dclo:
                    glob_is_close = True
                    print('restored simulation is undergoing CE')
                    print()

        sim.automateSimulationArchive(glob_archive, interval=1000.,deletefile=False)
        sim.heartbeat = heartbeat
    
    t0 = time.time()
        
    dt = 1
    
    while ((sim.t < tmax) and (glob_is_close == False) and (glob_is_eject == False)):
        try:
# this is pretty ugly but seems to be the only way I can abort on a close encounter.
# I guess that because the heartbeat function is called from the C code it can't pass exceptions
# back to the main Python program?
            sim.integrate(sim.t+dt)
        except rebound.Escape as error:
            print(error)
            sim.remove
            break
                
    t1 = time.time()
    
    if sim.t >= tmax - TINY:
        glob_is_stop = True
    
    print()
    print()
    print(f'{sim.t} years took {t1-t0} seconds')
    print()
    print()
    return
    
def setup_sim_moon(i):

    global glob_archive
    global glob_is_close
    global glob_is_eject
    global glob_is_stop
    global glob_dclo
    global glob_pl_done
    
    print()
    print(f'Simulation {i} with moons:')
    
    glob_archive = dir + f"planets_{i:04d}.bin"
    glob_is_close = False
    glob_is_eject = False
    glob_is_stop = False
    
    pl_done = (glob_archive in glob_pl_done)

    try:
        sim = rebound.Simulation(glob_archive)
    except:
        print(f'archive file {glob_archive} not found')
        return
    else:
#check end state
        if len(sim.particles) == 2:
            print('restored simulation has only one planet')
            print()
            glob_is_eject = True
        if sim.t >= tmax - TINY:
            print(f'restored simulation was stable for {tmax} yrs')
            print()
            glob_is_stop = True
        for p in sim.particles[1:]:
            rh2 = p.d**2 * (p.m/(3*sim.particles[0].m))**(2/3)
            for q in sim.particles[1:]:
                if p.hash.value == q.hash.value:
                    continue
                dx = p.x-q.x
                dy = p.y-q.y
                dz = p.z-q.z
                d2 = dx*dx + dy*dy + dz*dz
                if d2 <= rh2*glob_dclo:
                    glob_is_close = True
                    print('restored simulation is undergoing CE')
                    print()
        if (not glob_is_eject) and (not glob_is_stop) and (not glob_is_close) and (not pl_done):
            print('restored simulation needs continuing...')
            run_sim_pl(i)

# if CE, rewind and add moons

        rng = np.random.default_rng()
        
        has_moons = archive in glob_moons_added.keys()
        
        if glob_is_close and not has_moons:
            
            print('rewinding simulation...')
            sim.dt = -sim.dt
            tstart = sim.t - t_rewind
            sim.integrate(tstart)
            
            print('adding moons...')
            sim.dt = -sim.dt
            
            for p in sim.particles[1:]:
                p.r = (((9/4*np.pi)*p.m*u.Msun/(1.8*u.g/u.cm**3))**(1/3)).to_value(u.au)
                print(p.r)
            # moons. Data from JPL horizons 2022-01-26
            Nmoonsppl = 4 #per planet
            amo = (np.array([4.220279893619238E+05,6.712942692154385E+05,1.070941243876099E+06,
                             1.883987339695996E+06]) * u.km).to(u.au)
            emo = np.array([3.779947596328282E-03,9.515599858678114E-03,1.437934948643515E-03,
                            7.429672504931133E-03])
            imo = np.radians([2.233261880507145E+00,2.499911845043659E+00,2.320207222240029E+00,
                              1.964066225350317E+00])
            wmo = np.radians([2.230114025429823E+02,5.871258018281303E+01,3.520404542215327E+02,
                              3.411426411826523E+01]) # arg of peri
            Omo = np.radians([3.367638701787085E+02,3.289828238020597E+02,3.401167587299205E+02,
                              3.370179889677478E+02])
            mamo = np.radians([2.411569358232406E+02,1.160555854430442E+02,1.129206855751383E+02,
                               6.172934365391181E+01])
            name_mo_stem = ['Io','Europa','Ganymede','Callisto']
            Rmo = (np.array([1821.49,1560.8,2631.2,2410.3])*u.km).to(u.au)
            Mmo = (np.array([5959.9155,3202.7121,9887.8328,
                             7179.2834])*u.km**3/u.s**2 / c.G).to(u.Msun) #JPL gives product GM

            Omo_offset = np.radians(rng.uniform(0,360,2)) #we rotate Omega for all moons in the same moon system by the same amount
            name_moons = [[m+str(i+1) for m in name_mo_stem] for i in range(Npl)]

            for i in range(Npl):
                for j in range(Nmoonsppl):
                    sim.add(a=amo[j].to_value(u.au),e=emo[j],inc=imo[j],omega=wmo[j],Omega=Omo[j]+Omo_offset[i],
                            M=mamo[j],m=Mmo[j].to_value(u.Msun),r=Rmo[j].to_value(u.au),
                            primary=sim.particles[name_pl[i]],hash=name_moons[i][j])

            sim.move_to_com()
            
            # save a snapshot
            sim.simulationarchive_snapshot(glob_archive)
            
            with open('moons_added.txt','a') as f:
                f.write(f'{glob_archive} {sim.t}\n')
            
        else:
            print('No close encounter: passing to next simulation')
            return
        
#        # automate archive
#        sim.automateSimulationArchive(glob_archive, interval=1.,deletefile=False)
#        
#        tend = glob_moons_added[glob_archive] + tmoons
#        print('integrating with moons...')
#        clock_t0 = time.time()
#        sim_t0 = sim.t
#        sim.integrate(tend)
#        sim_t1 = sim.t
#        clock_t1 = time.time()
#        print(f'{sim_t1-sim_t0} years took {clock_t1-clock_t0} seconds')
#        
#        # final snapshot
#        sim.simulationarchive_snapshot(glob_archive)
            
        return

In [ ]:
glob_pl_done = []
glob_moons_added = {}

with open('planets_done.txt','r') as f:
    lines = f.readlines()

for l in lines:
    glob_pl_done.append(l.strip())

try:
    with open('moons_added.txt','r') as f:
        lines = f.readlines()

    for l in lines:
        s = l.split()
        glob_moons_added[s[0]] = float(s[1])
except:
    print('no simulations with moons yet added')
    
for i in range(Nsys):
    archive = dir + f"planets_{i:04d}.bin"

# if not in list of sims with planet bit finished, must continue 
    if not (archive in glob_pl_done):
        run_sim_pl(i)

    setup_sim_moon(i)

In [ ]:
try:
    with open('moons_added.txt','r') as f:
        lines = f.readlines()

    for l in lines:
        s = l.split()
        glob_moons_added[s[0]] = float(s[1])
except:
    print('no simulations with moons yet added')
else:
    for s in glob_moons_added.keys():

        if s == 'planet_test/planets_0098.bin':
            continue # this one has ground to a halt
        
        glob_log = s[:-3]+'log'
        
        glob_darr = [[[9999.9,9999.9,9999.9],[9999.9,9999.9,9999.9]],
                     [[9999.9,9999.9,9999.9],[9999.9,9999.9,9999.9]]]
        
        archive = s
        sim = rebound.Simulation(s)
        glob_planets = [n for n in name_pl]
        sim_hashes = [p.hash.value for p in sim.particles]
        glob_npl = sum([rebound.hash(p).value in sim_hashes for p in glob_planets])# this and the heartbeat needs to be done more carefully in case planets change their order

        tend = glob_moons_added[s] + t_moons
        sim.heartbeat = heartbeat
        sim.collision = "line"
        sim.collision_resolve = logged_merge
        sim.track_energy_offset = 0

        sim.automateSimulationArchive(s,interval=1.,deletefile=False)
        
        sim_t0 = sim.t
        clock_t0 = time.time()

        print(f'Simulation restored at {sim.t} years:    {s}')
        print()
        
        with open(glob_log,'a') as f:
            print(f'Simulation restored at {sim.t} years:    {s}',file=f)
            print('',file=f)

        
        while sim.t < tend:
            try:
                sim.integrate(tend)
            except rebound.Escape as error:
# save at this point
                sim.simulationarchive_snapshot(archive)

                print(error)
                hashes = set()
# Rebound example just allows one body to be removed; might have more    
                for h in name_all:
                    try:
                        p = sim.particles[h]
                        d2 = p.x**2 + p.y**2 + p.z**2
                        if d2 > sim.exit_max_distance**2:
                            hashes.add(h)
                    except:
                        if verbose:
                            print(f'{h} has already been removed')

# Here, we also want to remove any moons bound to a removed planet
                for h in name_all:
                    try:
                        if rebound.hash(find_primary(h,name_pl,sim)) in [ha.value for ha in hashes]:
                            hashes.add(h)
                    except:
                        if verbose:
                            print(f'{h} has already been removed')


                Ein = sim.calculate_energy()

                for h in hashes:
                    print(f'{h} ejected at {sim.t} years')
                    print(sim.particles[h])
                    with open(glob_log,'a') as f:
                        print(f'{h} ejected at {sim.t} years',file=f)
                        print(sim.particles[h],file=f)
                    if h in glob_planets:
                        glob_planets.remove(h)
                        glob_npl = glob_npl-1
                    sim.remove(hash=h)
                sim.move_to_com()    

                Eout = sim.calculate_energy()

#        if sim.track_energy_offset:
                sim.energy_offset += (Ein-Eout)

        sim_t1 = sim.t
        clock_t1 = time.time()
        
        print(f'{sim_t1-sim_t0} years took {clock_t1-clock_t0} seconds')
        print()
        
        with open(glob_log,'a') as f:
            print(f'{sim_t1-sim_t0} years took {clock_t1-clock_t0} seconds',file=f)
            print('',file=f)
        
        for moon in name_all:
            try:
                print(moon+' bound to '+find_primary(moon,name_pl,sim).hash)
                with open(glob_log,'a') as f:
                    print(moon+' bound to '+find_primary(moon,name_pl,sim).hash,file=f)
            except AttributeError:
                print(moon+' was removed')
                with open(glob_log,'a') as f:
                    print(moon+' was removed',file=f)

        print()
        print()
        
        with open(glob_log,'a') as f:
            print('',file=f)
            print('',file=f)